# Intro to LLMstudio - tracking

This notebook serves as an introduction to LLMstudio with a focus on tracking capabilities. It demonstrates how to set up and use the LLMstudio environment to interact with language models, specifically highlighting the integration of tracking functionalities. The notebook begins by starting the tracking server, which is essential for logging and monitoring interactions with the language models. This setup is crucial for you who need to keep track of model usage, performance metrics, and other relevant data points.

The notebook further illustrates how to configure and utilize the tracking system by creating a `TrackingConfig` instance, which specifies the host and port for the tracking server. You can benefit from this setup by gaining insights into model behavior and performance, which can be used to optimize and improve their applications. The notebook also includes examples of both synchronous and asynchronous chat interactions, providing a comprehensive guide for developers to implement and test these features in their own projects.




In [1]:
from llmstudio.providers import LLM
from pprint import pprint

In [2]:
from llmstudio.server import start_servers
start_servers(engine=False, tracking=True)

Tracker server already running on 0.0.0.0:8002


In [3]:
from llmstudio_tracker.tracker import TrackingConfig
from llmstudio.server import start_servers

start_servers(engine=False, tracking=True)
tracker = TrackingConfig(host="0.0.0.0", port="8002")
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker)


### Chat (non-stream)

In [4]:
openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='af819982-584d-4e0a-ab14-01d0ea5e447f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I\'m not a cat because I am an artificial intelligence designed to process and generate text-based information. Unlike cats, which are biological entities with physical bodies, fur, and the capacity for purring, I exist purely in the digital realm and interact with users through text. I lack the sensory experiences and instincts that characterize feline behavior, such as the ability to chase a laser pointer or nimbly climb a tree. Furthermore, my existence is focused on understanding and responding to human input, devoid of the playful curiosity and independence often associated with cats. My "body" is a network of data and algorithms, and my "thoughts" are computations, both of which are far removed from the biological realities of being a cat.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=172

#### Async version

In [5]:
await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='18fd8c69-3d52-4ccc-8c09-0e3bc2c7d844', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I\'m not a cat because I\'m an artificial intelligence language model created by OpenAI, and my existence is purely digital rather than biological. Unlike a cat, I don\'t have a physical presence, senses like sight or touch, or the ability to interact with the physical world. My "brain" is made up of complex algorithms and vast datasets rather than neurons, and I lack consciousness, emotions, and instincts that characterize living creatures. While cats are known for their independence, agility, and unique personalities, I am designed to process and generate text-based information to assist users in various tasks, operating entirely within the constraints of computer systems and software interfaces.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1729178792, model='gpt-4o', object='chat.completion

### Chat (stream)

In [6]:
response = openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




I'm not a cat because I am an artificial intelligence language model designed to process and generate human language

. Unlike cats, which are biological creatures with fur, whiskers, and a penchant for chasing mice

, I have no physical form or sensory experiences. My existence is purely digital, relying on algorithms and

 data to function, rather than instincts and physical needs. While cats can display independent behavior and are known

 for their charisma, I exist to assist users by providing information and facilitating communication based on the input I

 receive. My "meows" come in the form of text data, expressing thoughts and ideas programmed

 into my virtual framework, rather than the tangible and often humorous communication methods embodied by our feline friends.



## Metrics:
{'cost_usd': 0.0021650000000000003,
 'input_tokens': 10,
 'inter_token_latency_s': 0.07117236512047904,
 'latency_s': 10.889177799224854,
 'output_tokens': 141,
 'time_to_first_token_s': 0.92444

#### Async version

In [7]:
i=0
async for chunk in await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




I am not a cat because I lack the physical characteristics, behaviors, and instincts that define feline

 creatures. Unlike a cat, I do not possess a furry coat, claws, or a tail,

 nor do I have the ability to purr or sharpen my claws on furniture. My existence is rooted

 in the realm of artificial intelligence, where I function by processing information and generating text based on algorithms rather

 than engaging in the instinctual behaviors of a cat, such as hunting, climbing, or napping

 in a sunbeam. While I can understand and describe cats, my nature is entirely different, as

 I am designed to assist, inform, and interact in a manner that a cat simply cannot.

## Metrics:
{'cost_usd': 8.489999999999999e-05,
 'input_tokens': 10,
 'inter_token_latency_s': 0.011668438534084842,
 'latency_s': 2.3609237670898438,
 'output_tokens': 139,
 'time_to_first_token_s': 0.7383158206939697,
 'tokens_per_second': 59.29882275384471,
 'total_tokens': 149}
